In [11]:
#importing required libraries and functions
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

#loading data

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('data_1997_1998.csv')

#cleaning data
test_df=test_df.drop("Index",axis=1)
train_df=train_df.fillna({'zon.winds': train_df["zon.winds"].median(),'mer.winds': train_df["mer.winds"].median(),'humidity': train_df["humidity"].median(),'air temp.': train_df["air temp."].median(),'s.s.temp.':train_df["s.s.temp."].median()})
test_df=test_df.fillna({'zon.winds': test_df["zon.winds"].median(),'mer.winds': test_df["mer.winds"].median(),'humidity': test_df["humidity"].median(),'air temp.': test_df["air temp."].median()})


X_train = train_df.drop(columns=['s.s.temp.'])

#extracting the target variable
y_train = train_df['s.s.temp.']

X_test = test_df

#preprocessing data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#converting the data into numpy array to input into the LSTM model 
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))





KeyError: "['Index'] not found in axis"

In [2]:
#defining the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
from tensorflow.keras.layers import Dropout
model.add(Dropout(0.2))  
model.add(Dense(1))  

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

#storing the best model from 100 runs
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/100


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2611/2611 ━━━━━━━━━━━━━━━━━━━━ 2s 509us/step - loss: 562.2530 - val_loss: 33.3323
Epoch 2/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 486us/step - loss: 17.2914 - val_loss: 5.3395
Epoch 3/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 498us/step - loss: 6.8899 - val_loss: 4.3142
Epoch 4/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 494us/step - loss: 5.8301 - val_loss: 3.1841
Epoch 5/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 511us/step - loss: 4.7414 - val_loss: 2.2711
Epoch 6/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 496us/step - loss: 3.9280 - val_loss: 1.5435
Epoch 7/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 513us/step - loss: 3.2259 - val_loss: 1.0656
Epoch 8/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 522us/step - loss: 2.7982 - val_loss: 0.8140
Epoch 9/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 510us/step - loss: 2.6125 - val_loss: 0.7164
Epoch 10/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 505us/step - loss: 2.5633 - val_loss: 0.6699
Epoch 11/100
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 1s 503us/step - loss: 2.5357 - val_loss: 0.6593
Epoch

In [7]:
y_pred = model.predict(X_test_scaled)
test_df['s.s.temp.'] = y_pred

# Save the predictions to the CSV file
test_df.to_csv('data_1997_1998.csv', index=True)


1010/1010 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step


In [8]:
test_df

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.
0,1997,1,1,0.07,-110.0,-3.1,1.4,89.6,22.66,23.297033
1,1997,1,2,0.07,-110.0,-2.8,3.0,86.8,22.45,23.126827
2,1997,1,3,0.07,-110.0,-5.2,3.3,85.2,22.14,22.437290
3,1997,1,4,0.07,-110.0,-5.7,1.6,86.8,22.19,22.436567
4,1997,1,5,0.07,-110.0,-5.1,1.5,83.6,22.34,22.802656
...,...,...,...,...,...,...,...,...,...,...
32309,1998,6,11,8.96,-140.4,-5.1,-0.4,94.1,26.05,26.945679
32310,1998,6,12,8.96,-140.4,-4.3,-3.3,93.2,25.80,26.897392
32311,1998,6,13,8.95,-140.4,-6.1,-4.8,81.3,27.17,28.134768
32312,1998,6,14,8.96,-140.4,-4.9,-2.3,76.2,27.36,28.413458
